<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем библиотеки

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import re
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from lightgbm import LGBMClassifier 

import time
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Работаем с датасетом

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
print(df.info())
print('---------------------')
print(df.head())
print('---------------------')
print(df.isna().sum())
print('---------------------')
print(df.duplicated().sum())
print('---------------------')
df['toxic'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None
---------------------
   Unnamed: 0                                               text  toxic
0           0  Explanation\nWhy the edits made under my usern...      0
1           1  D'aww! He matches this background colour I'm s...      0
2           2  Hey man, I'm really not trying to edit war. It...      0
3           3  "\nMore\nI can't make any real suggestions on ...      0
4           4  You, sir, are my hero. Any chance you remember...      0
---------------------
Unnamed: 0    0
text          0
toxic         0
dtype: int64
---------------------
0
---------------------


0    143106
1     16186
Name: toxic, dtype: int64

Промежуточный вывод. Датасет содержит 159292 записи. Явных дубликатов и пропусков значений нет. Содержит 3 столбца - unnamed, text, toxic. В колонке text содержатся тексты комментариев, а в столбце toxic булевые значения является ли данный комментарий токсичным или нет (0- нет, 1 - да). Тексты комментариев на английском языке. Целевой признак - toxic. Столбец unnamed лишний, так как он дублирует индексы. 

Подготовим данные

In [4]:
df = df.drop(['Unnamed: 0'], axis=1)

Напишем функцию для обработки текстов комментариев c помощью регулярных выражений для дальнейшего исследования

In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [6]:
%%time
df['text'] = df['text'].map(lambda x: clean_text(x))

CPU times: user 8.37 s, sys: 93.5 ms, total: 8.46 s
Wall time: 8.46 s


In [7]:
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i am s...,0
2,hey man i am really not trying to edit war it ...,0
3,more i cannot make any real suggestions on imp...,0
4,you sir are my hero any chance you remember wh...,0


Вывод по п.1. Данные загружены, исследованы и подготовлены. Пропусков и явных дубликатов значений не выявлено. Столбец unnamed удален. Тексты комментариев обработаны с помощью регулярных выражений. Дисбаланс классов в целевом признаке. Дальше подумаем, что с этим можно сделать.

Напишем функцию РОS-тэгирования слов

In [8]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               
                "N": wordnet.NOUN,              
                "V": wordnet.VERB,              
                "R": wordnet.ADV               
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

Напишем функцию леммализации тектов постов

In [9]:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [10]:
%%time

df['text'] = df['text'].apply(lemm_text) 

CPU times: user 17min 8s, sys: 1min 39s, total: 18min 48s
Wall time: 18min 49s


## Обучение

Разделим данные на признаки (матрица X) и целевую переменную (y)

In [11]:
X = df.drop('toxic', axis=1)
y = df['toxic']

Разделим данные на обучающую и тестовую выборки

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)

Подберем параметры с помощью GridSearchCV с расчетом TF-IDF для моделей LogisticRegression и LGBMClassifier

Модель LogisticRegression. Добавил функцию получения лучшей метрики.

In [13]:
%%time

lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

params = {'clf__C': [0.1, 1, 10, 100],
          'clf__class_weight': ['balanced', None]}

lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=3, \
                       scoring='f1', n_jobs=-1, refit=False)
lr_grid.fit(X_train['text'], y_train)
lr_best_paramms = lr_grid.best_params_

print(lr_best_paramms)
print(lr_grid.best_score_)

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7658551782783536
CPU times: user 12min 11s, sys: 10min 16s, total: 22min 28s
Wall time: 22min 29s


 Модель LGBMClassifier. Добавил функцию получения лучшей метрики.

In [ ]:
%%time

lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=12345))])

params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [8, 10, -1]}

lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, \
                        scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(X_train['text'], y_train)
lgb_best_params = lgb_grid.best_params_

print(lgb_best_params)
print(lgb_grid.best_score_)

Векторизуем текст, преобразуем его в числовой вид

In [ ]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [ ]:
X_train = vectorize.fit_transform(X_train['text'])
X_test = vectorize.transform(X_test['text'])

**Тест модели LogisticRegression с лучшей метрикой F1 с подобранными гиперпараметрами**

Обучим модель  LogisticRegression, протестируем предсказания модели и полученим значение матрики качества F1 на тестовой выборке.

In [ ]:
lr_m = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
lr_m.fit(X_train, y_train)

test_pred = lr_m.predict(X_test)
test_f1 = f1_score(y_test, test_pred)

#def scoring(fitted_model):
    #test_pred = fitted_model.predict(X_test)
    #test_f1 = f1_score(y_test, test_pred)
    
print('F1 on test: {:.3f}'.format(test_f1))

Вывод по п.2. Выбраны модели LogisticRegression и LGBMClassifier для дальнейшего исследования. Подобраны лучшие гиперпараметры к моделям, проведено тестирование лучшей модели. Получена метрика качества модели на тестовой выборке. Она удовлетворяют условию поставленной задачи, лучшей является модель LogisticRegression с метрикой F1 = 0.785.

## Выводы

Общий вывод. 
При выполнении проекта были загружены, исследованы и подготовлены данные с разметкой о токсичности правок от интернет-магазина "Викишоп". Удалены лишние колонки, тексты комментариев обработаны с помощью регулярных выражений. Для обучения выбраны модели LogisticRegression и LGBMClassifier, к ним, с помощью GridSearchCV, подобраны лучшие гиперпараметры. Для тестирования выбрана модель LogisticRegression с лучшей метрикой F1. Полученный результат не меньше 0.75 как и было установлено заданием проекта. Рекомендую в качестве инструмента для поиска токсичных комментариев модель LogisticRegression.